<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=031f1d6fda4a70e41e3b03919dd0303e49be5f116378c33ef160782241736a40
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 13:10:17
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 87.05 K (0.63%)
Current PnL: -22.33 L (-14.88%)
CY Booked + Current PnL: -10.43 L (-6.95%)
-------------------
Total profit:  1.54 L
Total loss:  -23.87 L
-------------------
Total Booked + Current PnL: 16.55 L (13.44%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.55%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.52 L (64.33%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,98.51,54.0,M-SC,1.50,85491.0,-15286.0,15380.0,1.12,-15.17,17.99,0.09,245.0,-0.99,0.62,12.32,OX40N,NTT,DURABLES
18,CIPLA,1662.74,-20.47,54.0,X-LC,3.70,212405.0,7905.0,15399.0,-0.12,3.87,7.25,11.39,10.0,0.51,1.54,13.16,X40N,ATH,PHARMA
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.70,148141.0,12008.0,19125.0,7.93,8.82,12.91,22.86,91.0,0.63,1.07,22.55,X40,ATH,INSURANCE
83,VOLTAS,1530.00,0.18,55.0,X-MC,1.06,210105.0,18363.0,19393.0,0.65,9.58,9.23,19.69,99.0,0.95,1.52,17.03,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.97,215062.0,4696.0,24087.0,0.20,2.23,11.20,13.68,37.0,0.19,1.56,19.12,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.70,148141.0,12008.0,19125.0,7.93,8.82,12.91,22.86,91.0,0.63,1.07,22.55,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.59,61.0,X-LC,2.36,284287.0,18861.0,37924.0,3.35,7.11,13.34,21.40,11.0,0.50,2.06,14.22,XY25,NTT,FMCG
5,ASIANPAINT,3465.66,-17.24,45.0,X-LC,5.36,208877.0,-42891.0,96104.0,2.39,-17.04,46.01,21.13,27.0,-0.45,1.51,12.35,X40,ATH,PAINTS
84,WHIRLPOOL,2270.00,-47.73,38.0,M-SC,3.97,88425.0,-3074.0,81828.0,2.17,-3.36,92.54,86.07,223.0,-0.04,0.64,28.41,XR,NTT,DURABLES
43,JIOFIN,387.00,-1.23,55.0,X-LC,12.24,268406.0,1041.0,64417.0,2.13,0.39,24.00,24.48,48.0,0.02,1.94,55.58,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,ICICIPRULI,790.0,-23.52,38.0,X-MC,2.14,131532.0,-4256.0,47010.0,-2.68,-3.13,35.74,31.48,107.0,-0.09,0.95,8.60,X40,ATH,INSURANCE
15,CAMPUS,393.0,-23.92,54.0,M-SC,2.69,158410.0,-9365.0,65202.0,-2.02,-5.58,41.16,33.28,210.0,-0.14,1.15,22.78,XY24,NTT,FOOTWEAR
46,LAOPALA,464.0,95.46,30.0,H-SC,1.51,89376.0,-41729.0,92978.0,-1.65,-31.83,104.03,39.09,142.0,-0.45,0.65,15.11,AR,NTT,CERAMICS
7,ATULAUTO,844.0,3645.95,51.0,M-SC,4.75,168181.0,-28951.0,118786.0,-1.14,-14.69,70.63,45.57,236.0,-0.24,1.22,19.24,XY24,NTT,AUTO
39,INFY,2275.0,-19.60,47.0,X-LC,7.65,313856.0,968.0,170706.0,-1.10,0.31,54.39,54.87,3.0,0.01,2.27,6.98,X40,BTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.0,-186.36,68.0,M-SC,17.33,148689.0,-531.0,144704.0,0.27,-0.36,97.32,96.62,208.0,-0.0,1.08,60.91,XY25,NTT,FINANCE
59,SAIL,228.0,42.74,48.0,M-MC,12.29,224893.0,-69.0,167725.0,1.42,-0.03,74.58,74.53,192.0,-0.0,1.63,32.21,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,106.53,59.0,H-SC,11.52,130275.0,-12204.0,31722.0,1.41,-8.57,24.35,13.70,163.0,-0.38,0.94,53.76,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.51,54.0,M-SC,1.50,85491.0,-15286.0,15380.0,1.12,-15.17,17.99,0.09,245.0,-0.99,0.62,12.32,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,51.0,H-SC,1.86,222327.0,-47340.0,83684.0,0.65,-17.55,37.64,13.47,138.0,-0.57,1.61,13.05,XY24,NTT,PAINTS
17,CERA,9475.0,-23.39,43.0,H-SC,2.10,139840.0,-36063.0,78087.0,-0.18,-20.50,55.84,23.89,149.0,-0.46,1.01,20.77,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,43.0,H-MC,7.02,101926.0,-29909.0,72133.0,0.55,-22.69,70.77,32.03,98.0,-0.41,0.74,14.17,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.30,207762.0,4592.0,37543.0,2.01,2.26,18.07,20.74,55.0,0.12,1.50,7.93,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.19,69.0,M-LC,8.90,222220.0,17144.0,95421.0,1.03,8.36,42.94,54.89,52.0,0.18,1.61,34.47,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,56.0,M-LC,9.53,158725.0,17885.0,26602.0,1.90,12.70,16.76,31.59,66.0,0.67,1.15,42.55,XY24,NTT,BANKS
85,WIPRO,420.00,-11.98,54.0,M-LC,5.78,154461.0,3516.0,105945.0,0.29,2.33,68.59,72.51,53.0,0.03,1.12,8.49,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.74,194511.0,14703.0,97703.0,1.75,8.18,50.23,62.52,88.0,0.15,1.41,45.05,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,69.0,M-LC,8.90,222220.0,17144.0,95421.0,1.03,8.36,42.94,54.89,52.0,0.18,1.61,34.47,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.74,194511.0,14703.0,97703.0,1.75,8.18,50.23,62.52,88.0,0.15,1.41,45.05,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,54.0,M-LC,5.78,154461.0,3516.0,105945.0,0.29,2.33,68.59,72.51,53.0,0.03,1.12,8.49,XR,NTT,IT
36,INDIAMART,4810.62,-55.39,39.0,H-SC,8.29,124624.0,1288.0,130344.0,0.71,1.04,104.59,106.72,119.0,0.01,0.90,24.67,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-47.73,38.0,M-SC,3.97,88425.0,-3074.0,81828.0,2.17,-3.36,92.54,86.07,223.0,-0.04,0.64,28.41,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.33,142278.0,-134245.0,244320.0,-0.88,-48.55,171.72,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51520.0,-13486.0,164416.0,0.47,-20.75,319.13,232.18,198.0,-0.08,0.37,0.47,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.88,34.0,X-MC,1.51,228212.0,-19519.0,94571.0,0.95,-7.88,41.44,30.30,92.0,-0.21,1.65,4.37,X40,ATH,ELECTRICAL
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79806.0,-65354.0,141281.0,-0.59,-45.02,177.03,52.31,136.0,-0.46,0.58,6.82,X40N,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-15.75,36.0,X-MC,5.66,177514.0,-22992.0,120141.0,0.94,-11.47,67.68,48.45,90.0,-0.19,1.29,2.36,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.40,47.0,H-LC,0.94,157030.0,-29065.0,76552.0,1.07,-15.62,48.75,25.51,15.0,-0.38,1.14,15.72,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.18,55.0,X-MC,1.06,210105.0,18363.0,19393.0,0.65,9.58,9.23,19.69,99.0,0.95,1.52,17.03,XY25,NTT,AC
28,HAVELLS,2069.16,-7.88,34.0,X-MC,1.51,228212.0,-19519.0,94571.0,0.95,-7.88,41.44,30.30,92.0,-0.21,1.65,4.37,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.70,148141.0,12008.0,19125.0,7.93,8.82,12.91,22.86,91.0,0.63,1.07,22.55,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-7.59,61.0,X-LC,2.36,284287.0,18861.0,37924.0,3.35,7.11,13.34,21.40,11.0,0.50,2.06,14.22,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.33,142278.0,-134245.0,244320.0,-0.88,-48.55,171.72,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51520.0,-13486.0,164416.0,0.47,-20.75,319.13,232.18,198.0,-0.08,0.37,0.47,XY24,NTT,MISC
82,VBL,671.64,-21.55,39.0,X-LC,10.69,282634.0,-33208.0,145867.0,0.32,-10.51,51.61,35.67,5.0,-0.23,2.05,1.88,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.75,36.0,X-MC,5.66,177514.0,-22992.0,120141.0,0.94,-11.47,67.68,48.45,90.0,-0.19,1.29,2.36,X40N,ATH,FINANCE
41,ITC,452.00,-39.61,44.0,X-LC,2.82,195600.0,-4538.0,25428.0,0.81,-2.27,13.00,10.44,4.0,-0.18,1.42,3.17,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,31.0,X-SC,38.40,51520.0,-13486.0,164416.0,0.47,-20.75,319.13,232.18,198.0,-0.08,0.37,0.47,XY24,NTT,MISC
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79806.0,-65354.0,141281.0,-0.59,-45.02,177.03,52.31,136.0,-0.46,0.58,6.82,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.29,36.0,X-SC,4.41,93707.0,-34963.0,80260.0,0.43,-27.17,85.65,35.20,219.0,-0.44,0.68,7.55,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.16,50.0,X-SC,1.16,107926.0,5922.0,38001.0,1.97,5.81,35.21,43.06,122.0,0.16,0.78,23.92,X40N,ATH,MISC
32,HONAUT,58357.33,-23.25,44.0,X-SC,2.32,108135.0,-19803.0,66936.0,-0.12,-15.48,61.90,36.84,143.0,-0.30,0.78,10.70,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.97,45.0,X-LC,12.33,279133.0,-66823.0,133537.0,0.68,-19.32,47.84,19.28,1.0,-0.50,2.02,3.19,X40,ATH,IT
39,INFY,2275.00,-19.60,47.0,X-LC,7.65,313856.0,968.0,170706.0,-1.10,0.31,54.39,54.87,3.0,0.01,2.27,6.98,X40,BTT,IT
41,ITC,452.00,-39.61,44.0,X-LC,2.82,195600.0,-4538.0,25428.0,0.81,-2.27,13.00,10.44,4.0,-0.18,1.42,3.17,X40,NTT,FMCG
82,VBL,671.64,-21.55,39.0,X-LC,10.69,282634.0,-33208.0,145867.0,0.32,-10.51,51.61,35.67,5.0,-0.23,2.05,1.88,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,49.0,H-LC,7.41,249408.0,-12211.0,131413.0,1.49,-4.67,52.69,45.57,7.0,-0.09,1.81,6.13,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.67,80640.0,-32909.0,72963.0,-0.59,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
48,MASFIN,398.61,-21.41,38.0,H-SC,9.85,89280.0,-8700.0,30302.0,-0.67,-8.88,33.94,22.05,152.0,-0.29,0.65,29.77,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,106.53,59.0,H-SC,11.52,130275.0,-12204.0,31722.0,1.41,-8.57,24.35,13.70,163.0,-0.38,0.94,53.76,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,9.92,144544.0,-34527.0,100256.0,-0.60,-19.28,69.36,36.71,135.0,-0.34,1.05,43.42,XR,NTT,BANKS
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.70,148141.0,12008.0,19125.0,7.93,8.82,12.91,22.86,91.0,0.63,1.07,22.55,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,17.33,148689.0,-531.0,144704.0,0.27,-0.36,97.32,96.62,208.0,-0.00,1.08,60.91,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,9.92,144544.0,-34527.0,100256.0,-0.60,-19.28,69.36,36.71,135.0,-0.34,1.05,43.42,XR,NTT,BANKS
31,HINDZINC,730.22,36.19,69.0,M-LC,8.90,222220.0,17144.0,95421.0,1.03,8.36,42.94,54.89,52.0,0.18,1.61,34.47,X5K,ATH,METALS
4,ANGELONE,3033.00,12.76,65.0,X-SC,9.91,199555.0,8549.0,49150.0,1.21,4.48,24.63,30.21,157.0,0.17,1.45,26.29,X40N,NTT,FINANCE
50,PGHH,17907.65,-26.31,60.0,X-MC,3.05,213210.0,12390.0,55413.0,-0.22,6.17,25.99,33.76,80.0,0.22,1.54,11.69,X40,ATH,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.49
1,25,44.02
2,50,75.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.35
LC    33.74
MC    21.91
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.64
X40      14.81
XY25     12.18
XR       10.98
X40N     10.12
AR        8.24
OX40N     7.68
X200      1.81
X5K       1.61
SR        1.11
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.56
X-LC    22.57
X-MC    16.13
M-SC    12.71
M-LC     5.38
H-LC     4.76
X-SC     4.64
H-MC     3.84
M-MC     1.63
L-SC     1.44
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.09,-5.46,40.21
IT,12.99,-17.22,80.62
FINANCE,12.22,-12.10,62.91
MISC,6.92,-20.17,86.76
BANKS,6.45,-12.14,71.13
PAINTS,5.62,-18.97,36.70
ELECTRICAL,5.38,-10.85,49.74
INSURANCE,3.87,-3.41,42.08
AUTO,2.82,-47.56,118.11


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3342979.0,22
XR,1396408.0,14
AR,1192483.0,9
X40,791531.0,10
X40N,632017.0,8
OX40N,586284.0,10
XY25,500483.0,8
SR,267681.0,2
X5K,95421.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3471775.0,25
M-SC,1288896.0,15
X-LC,1123145.0,13
X-MC,1096869.0,11
X-SC,540044.0,6
H-MC,350402.0,3
H-LC,283296.0,3
M-LC,265511.0,4
L-SC,261757.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1208293.0      6
           XR         829442.0      7
           AR         820455.0      5
M-SC       XY24       699373.0      6
X-MC       XY24       587267.0      4
X-LC       X40        495152.0      5
           XY24       308737.0      2
X-SC       X40N       295368.0      4
H-SC       OX40N      274762.0      4
           SR         267681.0      2
X-LC       X40N       216508.0      3
X-MC       X40        216119.0      4
H-LC       AR         207965.0      2
H-MC       XY24       180566.0      1
X-MC       XY25       173342.0      2
L-SC       XR         171674.0      2
M-MC       XY24       167725.0      1
X-SC       XY24       164416.0      1
M-SC       AR         164063.0      2
           OX40N      149306.0      4
           XY25       144704.0      1
           XR         131450.0      2
X-MC       X40N       120141.0      1
M-LC       XR         105945.0      1
X-LC       XY25       102748.0      3
H-MC       XR          97703.0      1
M-LC       X5K         95421.0      1
L-SC       OX40N       90083.0      1
X-SC       X40         80260.0      1
H-LC       X200        75331.0      1
H-MC       OX40N       72133.0      1
H-SC       MH          71142.0      1
L-MC       XR          60194.0      1
L-LC       XY25        42146.0      1
M-LC       XY25        37543.0      1
           XY24        26602.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
